# Курсовой проект

**Основное**
- Дедлайн - 21 июня 23:59
- Целевая метрика precision@5. Порог для уcпешной сдачи проекта precision@5 > 25%
- Бейзлайн решения - [MainRecommender](https://github.com/geangohn/recsys-tutorial/blob/master/src/recommenders.py)
- Сдаем ссылку на github с решением. На github должен быть файл recommendations.csv (user_id | [rec_1, rec_2, ...] с рекомендациями. rec_i - реальные id item-ов (из retail_train.csv)

- Будет public тестовый датасет, на котором вы сможете измерять метрику
- Также будет private тестовый датасет для измерения финального качества
- НЕ обязательно, но крайне желательно использовать 2-ух уровневые рекоммендательные системы в проекте

**Hints:** 

Сначала просто попробуйте разные параметры MainRecommender:  
- N в топ-N товарах при формировании user-item матирцы (сейчас топ-5000)  
- Различные веса в user-item матрице (0/1, кол-во покупок, log(кол-во покупок + 1), сумма покупки, ...)  
- Разные взвешивания матрицы (TF-IDF, BM25 - у него есть параметры)  
- Разные смешивания рекомендаций (обратите внимание на бейзлайн - прошлые покупки юзера)  

Сделайте MVP - минимально рабочий продукт - (пусть даже top-popular), а потом его улучшайте

Если вы делаете двухуровневую модель - следите за валидацией 

## Библиотеки и загрузка данных

In [1]:
import warnings
warnings.simplefilter(action='ignore')

import pandas as pd

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.metrics import total_precision_at_N
from src.utils import prefilter_items, split_train_val, prepare_users, prepare_items, unstack_user_item
from src.recommenders import RecommenderDataset, BaseRecommender, OwnRecommender, AlsRecommender, Level2Recommender

Using TensorFlow backend.


In [2]:
DATAROOT = './data/'

data = pd.read_csv(f'{DATAROOT}transactions.csv.gz', compression='gzip')
test = pd.read_csv(f'{DATAROOT}test.csv.gz', compression='gzip')

items = pd.read_csv(f'{DATAROOT}product.csv.gz')
items.columns = [col.lower() for col in items.columns]
items.rename(columns={'product_id': 'item_id'}, inplace=True)
items = prepare_items(items)

users = pd.read_csv(f'{DATAROOT}demographic.csv.gz')
users.columns = [col.lower() for col in users.columns]
users.rename(columns={'household_key': 'user_id'}, inplace=True)
users = prepare_users(users)

## Разбивка и обучение моделей

In [3]:
# делим на train / validate1 / validate2
train, val1, val2 = split_train_val(data, 6, 3)

# фильтруем items от ненужного и берем только 5000 (+1 на остальное)
was = train.item_id.nunique()
train = prefilter_items(train, price=(None, None), popular=(None, None), products=items, top_n=5000)
print(f'Decreased amount of items from {was} to {train.item_id.nunique()}')

Decreased amount of items from 83685 to 5001


### Расчет моделей первого уровня

In [5]:
ds = RecommenderDataset(train)

base = BaseRecommender(ds).fit()
own = OwnRecommender(ds).fit()
als = AlsRecommender(ds).fit()

def recommend(data, N=50):
    res = data.groupby('user_id')['item_id'].unique().reset_index()
    res.columns=['user_id', 'actual']

    res['recommend_base'] = base.recommend(res.user_id, N)
    res['recommend_own'] = own.recommend(res.user_id, N)
    res['recommend_als'] = als.recommend(res.user_id, N)
    res['recommend_als_similar_users'] = als.recommend(res.user_id, N, by='similarUsers')
    res['recommend_als_similar_items'] = als.recommend(res.user_id, N, by='similarItems')
    return res

result = pd.DataFrame(columns=['train', 'valid', 'test'])

print('Оценка TRAIN')
result['train'] = total_precision_at_N(recommend(train, 5), 5)

print('Оценка VALID')
val1_candidates = recommend(val1)
val2_candidates = recommend(val2)
result['valid'] = total_precision_at_N(val1_candidates, 5)

print('Оценка TEST')
test_candidates = recommend(test)
result['test'] = total_precision_at_N(test_candidates, 5)

result


Оценка TRAIN
Оценка VALID
Оценка TEST


,train,valid,test
precision_base,0.986950,0.430734,0.339098
precision_als_similar_users,0.625861,0.270659,0.213475
precision_als_similar_items,0.463571,0.095636,0.068541
precision_als,0.395116,0.125534,0.102175
precision_own,0.038911,0.007057,0.003395


### Расчет моделей второго уровня

In [6]:
# подготовим dataframes с кандидатами
model1_candidates = 'recommend_base'

train_lv2 = unstack_user_item(val1_candidates, model1_candidates, 'actual')
valid_lv2 = unstack_user_item(val2_candidates, model1_candidates, 'actual')
test_lv2  = unstack_user_item(test_candidates, model1_candidates, 'actual')

# Обучаем на кандидатах для VAL-1
Level2 = Level2Recommender(transactions=data, 
                           items=items, items_emb=als.items_embedings(),
                           users=users, users_emb=als.users_embedings())

Level2.fit_predict_report(train_lv2)

def recommend(data, candidates, N=5):
    df = Level2.recommend(data, N)
    tmp = candidates.merge(df, how='left', on='user_id')
    return tmp[['user_id', 'actual', 'recommend_level2']]

result2 = pd.DataFrame(columns=['train', 'valid', 'test'])

print('Оценка TRAIN (для этого уровня это VAL-1)')
result2['train'] = total_precision_at_N(recommend(train_lv2, val1_candidates))

print('Оценка VALID (для этого уровня это VAL-2)')
result2['valid'] = total_precision_at_N(recommend(valid_lv2, val2_candidates))

print('Оценка TEST')
result2['test'] = total_precision_at_N(recommend(test_lv2, test_candidates))

pd.concat([result, result2])

Fitting ...done
Качество на TRAIN:
              precision    recall  f1-score   support

         0.0       0.88      0.98      0.93     58472
         1.0       0.86      0.39      0.54     13328

    accuracy                           0.88     71800
   macro avg       0.87      0.69      0.73     71800
weighted avg       0.87      0.88      0.86     71800

Качество на TEST:
              precision    recall  f1-score   support

         0.0       0.84      0.95      0.89     29261
         1.0       0.51      0.23      0.31      6639

    accuracy                           0.82     35900
   macro avg       0.68      0.59      0.60     35900
weighted avg       0.78      0.82      0.79     35900

Оценка TRAIN (для этого уровня это VAL-1)
Оценка VALID (для этого уровня это VAL-2)
Оценка TEST


,train,valid,test
precision_base,0.986950,0.430734,0.339098
precision_als_similar_users,0.625861,0.270659,0.213475
precision_als_similar_items,0.463571,0.095636,0.068541
precision_als,0.395116,0.125534,0.102175
precision_own,0.038911,0.007057,0.003395
precision_level2,0.021170,0.044956,0.041273


# Вывод

> **Как я не пытался у меня не получилось лучшего качества чем я получил на модели первого уровня когда я просто рекомендую популярные или те которые пользователь до этого покупал**

> Модель второго уровня почти всегда только усугубляет ситуацию, видимо я что-то всеже упускаю из виду

# Итоговая модель

In [7]:
# кандидаты

def level1(data):
    df = data[['user_id', 'item_id']].drop_duplicates()
    df = df.groupby('user_id')['item_id'].unique().reset_index()
    df.columns=['user_id', 'actual']
    df['recommend_main'] = base.recommend(df.user_id, 50)
    print('\n >> Level1 Precision@5:')
    print(total_precision_at_N(df))
    lv2 = unstack_user_item(df, 'recommend_main', 'actual')
    return df, lv2

def level2(data, candidates):
    tmp = Level2.recommend(data, 5)
    tmp = candidates.merge(tmp, how='left', on='user_id')
    print('\n >> Level2 Precision@5:\n')
    print(total_precision_at_N(tmp))
    return tmp
    
def model(data, fit=False):
    candidates, lv2 = level1(data)
    if fit:
        Level2.fit_predict_report(lv2)
    return level2(lv2, candidates)

print('\n---- Качество на TRAIN-e ----')
main = model(data, True)

print('\n---- Качество на TEST-e ----')
_ = model(test)


---- Качество на TRAIN-e ----

 >> Level1 Precision@5:
precision_main    0.987275
dtype: float64
Fitting ...done
Качество на TRAIN:
              precision    recall  f1-score   support

         0.0       0.96      0.99      0.98      5079
         1.0       1.00      1.00      1.00     78221

    accuracy                           1.00     83300
   macro avg       0.98      1.00      0.99     83300
weighted avg       1.00      1.00      1.00     83300

Качество на TEST:
              precision    recall  f1-score   support

         0.0       0.89      0.94      0.91      2548
         1.0       1.00      0.99      0.99     39102

    accuracy                           0.99     41650
   macro avg       0.94      0.96      0.95     41650
weighted avg       0.99      0.99      0.99     41650


 >> Level2 Precision@5:

precision_main      0.987275
precision_level2    0.889156
dtype: float64

---- Качество на TEST-e ----

 >> Level1 Precision@5:
precision_main    0.339098
dtype: float64

## Сохранение модели

In [8]:
def save(data, field, file):
    tmp = data[['user_id', field]].copy()
    tmp.columns = ['user_id', 'result']
    tmp['result'] = tmp.result.apply(lambda x: list(x)[:5])
    tmp.to_csv(f'{file}.csv')
    tmp.to_pickle(f'{file}.pkl')

    
# сохраним предсазания модели первого уровня
save(main, 'recommend_main', 'level1')

# сохраним предсазания модели второго уровня
save(main, 'recommend_level2', 'recommendations')